In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
import time
from collections import defaultdict
import itertools

In [2]:
## custom-defined layers

class LocationAdd(layers.Layer):
    def __init__(self, input_dim):
        super(LocationAdd, self).__init__()
        w_init = tf.keras.initializers.GlorotNormal()
        self.w = tf.Variable(initial_value=w_init(shape=(input_dim,)), trainable=True)

    def call(self, inputs):
        return tf.add(inputs, self.w)
    
class RegressionAdd(layers.Layer):
    def __init__(self, input_dim):
        super(RegressionAdd, self).__init__()
        self.input_dim = input_dim
        w_init = tf.keras.initializers.GlorotNormal()
        self.w = tf.Variable(initial_value=w_init(shape=(input_dim-1,)), trainable=True)
    
    def call(self, inputs):
        input_X, input_z = tf.split(inputs, [self.input_dim-1, 1], axis=1)
        return tf.concat([input_X, tf.add(tf.reshape(tf.linalg.matvec(input_X, self.w),[-1,1]),input_z)], axis=1)

In [3]:
## custom-defined constraints

from tensorflow.python.keras import backend as K
from tensorflow.python.ops import math_ops
class Constraint(object):
    def __call__(self, w):
        return w
    def get_config(self):
        return {}
    
class Max1Norm(Constraint):
    """1-Norm weight constraint.
    """

    def __init__(self, max_value=2, axis=0):
        self.max_value = max_value
        self.axis = axis

    def __call__(self, w):
        norms = math_ops.reduce_sum(math_ops.abs(w), axis=self.axis, keepdims=True)
        desired = K.clip(norms, 0, self.max_value)
        return w * (desired / (K.epsilon() + norms))

    def get_config(self):
        return {'max_value': self.max_value, 'axis': self.axis}

In [4]:
class WganError(tf.keras.losses.Loss):
    def call(self, y_true, y_pred):
        #y_pred = ops.convert_to_tensor(y_pred)
        #y_true = math_ops.cast(y_true, y_pred.dtype)
        return K.mean(y_pred - y_true, axis=-1)

In [50]:
class WGAN:
    
    ''' A static model, with fixed input size.
        Model should not be defined in train(), so it should not depend on dataset dimension.
        Model API has an advantage that it can save weights between different calls of train.
        Instead of using tf.Session() as before, where only one training can happen, next will refresh,
        using Model() API avoids this, it provides a model which saves weights outside tf.Session()!
    '''
    
    def __init__(self, dim_x, target):
        self.dim_x = dim_x
        self.target = target
        self.generator = self.generator_model(dim_x, target)
        self.discriminator = self.discriminator_model(dim_x)

    
    def generator_model(self, dim, target):
        if target == "location":
            inputs = tf.keras.Input(shape=(dim,))
            out = LocationAdd(dim)(inputs)
            model = tf.keras.Model(inputs=inputs, outputs=out)
            return model
        elif target == "cov-matrix":
            inputs = tf.keras.Input(shape=(dim,))
            out = layers.Dense(units=dim, use_bias=False)(inputs)
            model = tf.keras.Model(inputs=inputs, outputs=out)
            return model
        elif target == "regression":
            inputs = tf.keras.Input(shape=(dim,))
            out = RegressionAdd(dim)(inputs)
            model = tf.keras.Model(inputs=inputs, outputs=out)
            return model
    
    def discriminator_model(self, dim):        
        inputs = tf.keras.Input(shape=(dim,))
        dense1 = layers.Dense(units=2*dim, activation=tf.keras.activations.sigmoid, 
                              kernel_constraint=tf.keras.constraints.MaxNorm(max_value=1, axis=0))(inputs)
        dense2 = layers.Dense(units=dim, activation=tf.keras.activations.relu, 
                              kernel_constraint=tf.keras.constraints.MaxNorm(max_value=1, axis=0))(dense1)
        out = layers.Dense(units=1, activation=None, 
                           kernel_constraint=Max1Norm(max_value=1,axis=0))(dense2)
        model = tf.keras.Model(inputs=inputs, outputs=out)
        return model
    
    @staticmethod
    def discriminator_loss(real_output, fake_output):
        #cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
        #real_loss = cross_entropy(tf.ones_like(real_output), real_output)
        #fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
        #total_loss = real_loss + fake_loss
        total_loss = WganError()(real_output, fake_output)
        return total_loss
    
    @staticmethod
    def generator_loss(fake_output):
        # cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
        # loss = cross_entropy(tf.ones_like(fake_output), fake_output)
        loss = WganError()(fake_output, tf.zeros_like(fake_output))
        return loss
    
    
    def train(self, dataset, epochs, batch_size, step_size):
        self.generator_optimizer = tf.keras.optimizers.RMSprop(step_size)
        self.discriminator_optimizer = tf.keras.optimizers.RMSprop(step_size)
        for epoch in range(epochs):
            start = time.time()
            for i in range(dataset.shape[0]//batch_size):
                noise = tf.random.normal([batch_size, self.dim_x])
                with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
                    generated = self.generator(noise, training=True)
                    real_output = self.discriminator(dataset[i*batch_size:(i+1)*batch_size], training=True)
                    fake_output = self.discriminator(generated, training=True)

                    gen_loss = WGAN.generator_loss(fake_output)
                    disc_loss = WGAN.discriminator_loss(real_output, fake_output)

                    gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
                    gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

                    self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
                    self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))

#             print('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
#             A = self.generator.trainable_variables[0].numpy()
#             if self.target == "location" or self.target == "regression":
#                 print(A)
#             elif self.target == "cov-matrix":
#                 sigma_hat = np.matmul(A, A.T)
#                 print(sigma_hat)
#                 print(np.linalg.norm(sigma_hat-np.identity(self.dim_x), ord=2))
#             print("generator loss:", gen_loss.numpy(), "discriminator loss: ", disc_loss.numpy())
#             print(np.linalg.norm(self.discriminator.trainable_variables[0].numpy(), ord=1, axis=0))

## Covariance matrix estimation

https://stackoverflow.com/questions/56201185/how-to-find-a-variable-by-name-in-tensorflow2-0

In [60]:
def simulate_cov(N, p, model):    
    A = np.random.uniform(size=(p,p))
    cov = A.T @ A
    data = np.random.normal(size=(N, p)) @ A
    z = np.random.binomial(n=1,p=0.1,size=(N,1))
    if model == "Cauchy":
        noise = np.random.standard_cauchy(size=(N,p))
    elif model == "Normal":
        noise = np.random.normal(5,size=(N,p))
    elif model == "Gumbel":
        noise = np.random.gumbel(size=(N,p)) * 5
    data_perturbed = data * (1-z) + noise * z
    data_perturbed = data_perturbed.astype(np.float32)
    # print("sample covariance: \n", np.cov(data_perturbed.T))
    # print("true covariance: \n", cov)
    
    batch_size = 32 if N == 100 else 64
    epochs = 200
    step_size = 0.01
    wgan = WGAN(dim_x=p, target="cov-matrix")
    wgan.train(data_perturbed, epochs=epochs, batch_size=batch_size, step_size=step_size)
    
    sample_2norm_error = np.round(np.linalg.norm(np.cov(data_perturbed.T)-cov, ord=2), 4)
    res_sample_cov[(N,p,model)].append(sample_2norm_error)
    Ahat = wgan.generator.trainable_variables[0].numpy()
    wgan_error = np.round(np.linalg.norm(Ahat.T@Ahat-cov, ord=2), 4)
    res_wgan_cov[(N,p,model)].append(wgan_error)

In [ ]:
Ns = [100, 1000, 5000, 10000]
ps = [10, 20, 40]
models = ["Cauchy", "Normal", "Gumbel"]
res_sample_cov = defaultdict(list)
res_wgan_cov = defaultdict(list)
outfile = open("cov_est.txt", 'w')

for N, p, model in itertools.product(Ns, ps, models):
    t = time.time()
    print("N={}, p={}, model={} in progress.".format(N, p, model))
    
    for i in range(10):
        simulate_cov(N, p, model)
    outfile.write("N={}, p={}, model={} samp: {}\n".format(N, p, model, res_sample_cov[(N,p,model)]))
    outfile.write("N={}, p={}, model={} wgan: {}\n".format(N, p, model, res_wgan_cov[(N,p,model)]))
    outfile.flush()
    
    print("N={}, p={}, model={} in done.({:.2f} minutes)".format(N, p, model, (time.time()-t)/60))

In [ ]:
# res_sample_cov = defaultdict(list)
# res_wgan_cov = defaultdict(list)
# simulate_cov(100, 10, "Cauchy")
# print(res_sample_cov)
# print(res_wgan_cov)

In [ ]:
# N = 1000
# p = 4

# A = np.random.uniform(size=(p,p))
# cov = A.T @ A

# data = np.random.normal(size=(N,p)) @ A
# noise = np.random.standard_cauchy(size=(N,p))
# #noise = np.random.gumbel(size=(N,p)) * 5
# z = np.random.binomial(n=1,p=0.1,size=(N,1))
# data_perturbed = data * (1-z) + noise * z
# data_perturbed = data_perturbed.astype(np.float32)
# print("sample covariance: \n", np.cov(data_perturbed.T))
# print("true covariance: \n", cov)

In [51]:
# epochs = 100
# batch_size = 32
# step_size = 0.01

# wgan = WGAN(dim_x=p, target="cov-matrix")
# wgan.train(data_perturbed, epochs=epochs, batch_size=batch_size, step_size=step_size)

In [ ]:
# print("2-norm loss, samp: ", np.linalg.norm(np.cov(data_perturbed.T)-cov, ord=2))
# Ahat = wgan.generator.trainable_variables[0].numpy()
# print("2-norm loss, wgan: ", np.linalg.norm(Ahat.T@Ahat - cov, ord=2))
# print("cov hat: \n", A.T @ A)

## Location Estimation

In [10]:
# def simulate_loc(N, p, model):    
#     theta = np.repeat(np.array([1,2,3,4,5]), p//5)
#     data = np.random.normal(size=(N, p)) + theta
#     # print("sample mean: \n", np.mean(data, axis=0))
#     z = np.random.binomial(n=1,p=0.1,size=(N,1))
#     if model == "Cauchy":
#         noise = np.random.standard_cauchy(size=(N,p))
#     elif model == "Normal":
#         noise = np.random.normal(2,size=(N,p))
#     elif model == "Gumbel":
#         noise = np.random.gumbel(size=(N,p))
#     # A = np.random.uniform(size=(p,p))
#     # noise = np.random.normal(size=(N,p)) @ A
#     data_perturbed = data * (1-z) + noise * z
#     data_perturbed = data_perturbed.astype(np.float32)
#     # print("noisy sample mean: \n", np.mean(data_perturbed, axis=0))
    
#     batch_size = 32 if N == 100 else 64
#     epochs = 200
#     step_size = 0.01
#     wgan = WGAN(dim_x=p, target="location")
#     wgan.train(data_perturbed, epochs=epochs, batch_size=batch_size, step_size=step_size)
#     wgan.train(data_perturbed, epochs=epochs//4, batch_size=batch_size, step_size=step_size/2)
#     wgan.train(data_perturbed, epochs=epochs//4, batch_size=batch_size, step_size=step_size/10)
    
#     sample_mean_error = np.round(np.linalg.norm(np.mean(data_perturbed, axis=0)-theta)**2, 4)
#     res_sample_mean[(N,p,model)].append(sample_mean_error)
#     wgan_error = np.round(np.linalg.norm(wgan.generator.trainable_variables[0].numpy()-theta)**2, 4)
#     res_wgan_mean[(N,p,model)].append(wgan_error)

In [ ]:
# Ns = [100, 1000, 5000, 10000]
# ps = [10, 20, 40, 80]
# models = ["Cauchy", "Normal", "Gumbel"]
# res_sample_mean = defaultdict(list)
# res_wgan_mean = defaultdict(list)
# outfile = open("location_est.txt", 'w')

# for N, p, model in itertools.product(Ns, ps, models):
#     t = time.time()
#     print("N={}, p={}, model={} in progress.".format(N, p, model))
    
#     for i in range(10):
#         simulate_loc(N, p, model)
#     outfile.write("N={}, p={}, model={} samp: {}\n".format(N, p, model, res_sample_mean[(N,p,model)]))
#     outfile.write("N={}, p={}, model={} wgan: {}\n".format(N, p, model, res_wgan_mean[(N,p,model)]))
#     outfile.flush()
    
#     print("N={}, p={}, model={} in done.({:.2f} minutes)".format(N, p, model, (time.time()-t)/60))

In [ ]:
# res_sample_mean = defaultdict(list)
# res_wgan_mean = defaultdict(list)
# simulate_loc(100, 80, "Gumbel")

## Regression

In [ ]:
# N = 1000
# p = 4
# sigma = 1
# beta = np.array([1,2,3,4], dtype=np.float32)

# data_X = np.random.normal(size=(N,p)).astype(np.float32)
# data_y = data_X @ beta + np.random.normal(scale=sigma, size=(N,)).astype(np.float32)
# data_y = data_y.reshape([-1,1])
# data_reg = np.concatenate([data_X, data_y], axis=1)

In [ ]:
# betahat = np.linalg.solve(data_X.T@data_X, (data_X.T@data_y))
# print("least square estimate: \n", betahat)

In [ ]:
# epochs = 10
# batch_size = 32
# step_size = 0.01

# wgan = WGAN(dim_x=p+1, target="regression")
# wgan.train(data_reg, epochs=epochs, batch_size=batch_size, step_size=step_size)

In [ ]:
# wgan.train(data, epochs=10, batch_size=32, step_size=0.001)

## Miscellaneous

In [ ]:
# correct test version of model with self defined layers
# def build_model():
#     a = tf.keras.Input(shape=(4,))
#     out = LocationAdd(input_dim=4)(a+5)
#     model = tf.keras.Model(inputs=a, outputs=out)
#     return model
# model = build_model()
# model2 = build_model()
# print(model.trainable_variables)
# print(model2.trainable_variables)
# model.compile(optimizer='rmsprop', loss=tf.keras.losses.MeanSquaredError())
# model.fit(x=data,y=data, batch_size=1, epochs=100)
# print(model.trainable_variables)
# print(model2.trainable_variables)

## tf.keras.layers.add can make variables not trainable, below is not correct
# a = tf.keras.Input(shape=(4,))
# b = tf.Variable(initial_value=tf.random_normal_initializer()(shape=(4,)), trainable=True)
# out = tf.keras.layers.add([a+5,b])